In [1]:
import httpx
import pandas as pd
import requests 

from bs4 import BeautifulSoup


display.max_colwidth = 100

df_full = pd.read_csv('data/tweets_users_sept_21.csv', parse_dates=['date'])

In [2]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   url_x            2973 non-null   object             
 1   date             2973 non-null   datetime64[ns, UTC]
 2   content          2973 non-null   object             
 3   renderedContent  2973 non-null   object             
 4   id_x             2973 non-null   int64              
 5   user             2973 non-null   object             
 6   outlinks         2973 non-null   object             
 7   tcooutlinks      2973 non-null   object             
 8   replyCount       2973 non-null   int64              
 9   retweetCount     2973 non-null   int64              
 10  likeCount        2973 non-null   int64              
 11  quoteCount       2973 non-null   int64              
 12  conversationId   2973 non-null   int64              
 13  lang             2

In [3]:
def cleaner(text):
    return text.replace("['", "").replace("']", '').replace("[]", "")

In [4]:
df_full['outlinks_cleaned'] = df_full["outlinks"].apply(lambda x: cleaner(x))

In [5]:
df_full["outlinks_cleaned"].value_counts()

                                                                                                                            991
http://www.podrevday.com                                                                                                     71
http://www.PodRevDay.com                                                                                                     36
https://www.stephfuccio.com/podrevday-podcast-review-day.html                                                                25
http://podrevday.com                                                                                                         14
                                                                                                                           ... 
https://www.podchaser.com/podcasts/game-for-a-movie-1121022/reviews/56186                                                     1
https://twitter.com/Podhero/status/1368940145047711746                                                  

In [6]:
podchaser_reviews = df_full[df_full["outlinks_cleaned"].str.startswith("https://www.podchaser")]
len(podchaser_reviews)

1252

In [7]:
pd.set_option('display.max_colwidth', 100)

podchaser_reviews.loc[:5, ['outlinks', 'outlinks_cleaned']]

,outlinks,outlinks_cleaned
0,['https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785/reviews/70541'],https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785/reviews/70541
2,['https://www.podchaser.com/podcasts/but-it-was-aliens-1096389/reviews/70534'],https://www.podchaser.com/podcasts/but-it-was-aliens-1096389/reviews/70534
4,['https://www.podchaser.com/podcasts/the-secret-sits-1603491/reviews/70522'],https://www.podchaser.com/podcasts/the-secret-sits-1603491/reviews/70522


In [8]:
def splitter(text):
    return "/".join(text.split('/', 5)[:5])

In [9]:
podchaser_reviews['podcast_url'] = podchaser_reviews["outlinks_cleaned"].apply(lambda x: splitter(x))

/tmp/ipykernel_6336/2943266381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  podchaser_reviews['podcast_url'] = podchaser_reviews["outlinks_cleaned"].apply(lambda x: splitter(x))


In [10]:
podchaser_reviews.tail()

,url_x,date,content,renderedContent,id_x,user,outlinks,tcooutlinks,replyCount,retweetCount,...,friendsCount,url_y,verified,geotext,city,country,name,iso3,outlinks_cleaned,podcast_url
2943,https://twitter.com/unstructuredp/status/1226177809984446464,2020-02-08 16:15:45+00:00,Check out my ⭐️⭐️⭐️⭐️⭐️ review of Hunting Warhead on @Podchaser https://t.co/csJiK2G3ZO\n#PodRevDay,Check out my ⭐️⭐️⭐️⭐️⭐️ review of Hunting Warhead on @Podchaser podchaser.com/podcasts/hunti…\n#...,1226177809984446464,"{'username': 'unstructuredp', 'displayname': 'Unstructured', 'id': 905937139631783937, 'descript...",['https://www.podchaser.com/podcasts/hunting-warhead-936830/reviews/21414'],['https://t.co/csJiK2G3ZO'],0,4,...,9962,https://twitter.com/unstructuredp,False,<geotext.geotext.GeoText object at 0x7f784afa80a0>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/hunting-warhead-936830/reviews/21414,https://www.podchaser.com/podcasts/hunting-warhead-936830
2944,https://twitter.com/MarkCockram/status/1226175607186296832,2020-02-08 16:07:00+00:00,I left a review for those lovely @FTEpodcast boys!\nhttps://t.co/MGKjwK21Go #PodRevDay https://t...,I left a review for those lovely @FTEpodcast boys!\npodchaser.com/podcasts/fligh… #PodRevDay twi...,1226175607186296832,"{'username': 'MarkCockram', 'displayname': 'The Human Palindrome', 'id': 21389716, 'description'...","['https://www.podchaser.com/podcasts/flight-through-entirety-a-doct-2663/reviews/21373', 'https:...","['https://t.co/MGKjwK21Go', 'https://t.co/6Ras8iBOPu']",2,2,...,531,https://twitter.com/MarkCockram,False,<geotext.geotext.GeoText object at 0x7f784df76640>,NaN,United Kingdom,United Kingdom,GBR,"https://www.podchaser.com/podcasts/flight-through-entirety-a-doct-2663/reviews/21373', 'https://...",https://www.podchaser.com/podcasts/flight-through-entirety-a-doct-2663
2951,https://twitter.com/OSWpodcast1/status/1226140733817737217,2020-02-08 13:48:26+00:00,#PodRevDay check out @CagesKiss \n\n@Podchaser https://t.co/GebmbQ4TzB,#PodRevDay check out @CagesKiss \n\n@Podchaser podchaser.com/podcasts/cages…,1226140733817737217,"{'username': 'OSWpodcast1', 'displayname': '🎬On Second Watch🎙Podcast', 'id': 1217819737385816066...",['https://www.podchaser.com/podcasts/cages-kiss-802419/reviews/21407'],['https://t.co/GebmbQ4TzB'],2,5,...,2707,https://twitter.com/OSWpodcast1,False,<geotext.geotext.GeoText object at 0x7f784e01e820>,Michigan,United States,United States,USA,https://www.podchaser.com/podcasts/cages-kiss-802419/reviews/21407,https://www.podchaser.com/podcasts/cages-kiss-802419
2959,https://twitter.com/stephfuccio/status/1225692286010290186,2020-02-07 08:06:27+00:00,I respect the heck out of everyone who spreads #podlove so moving up my #PodRevDay review for #P...,I respect the heck out of everyone who spreads #podlove so moving up my #PodRevDay review for #P...,1225692286010290186,"{'username': 'stephfuccio', 'displayname': 'Stephanie Fuccio', 'id': 153349145, 'description': '...",['https://www.podchaser.com/podcasts/0/episodes/113-zaltzology-52130629/reviews/21345'],['https://t.co/S5IIjzRJ3p'],2,3,...,3132,https://twitter.com/stephfuccio,False,<geotext.geotext.GeoText object at 0x7f784e3adfd0>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/0/episodes/113-zaltzology-52130629/reviews/21345,https://www.podchaser.com/podcasts/0
2971,https://twitter.com/stephfuccio/status/1222582778417557506,2020-01-29 18:10:23+00:00,The next #PodRevDay is February 8th but can't wait that long! Discovered this #expatpodcast &amp...,The next #PodRevDay is February 8th but can't wait that long! Discovered this #expatpodcast &amp...,1222582778417557506,"{'username': 'stephfuccio', 'displayname': 'Stephanie Fuccio', 'id': 153349145, 'description': '...",['https://www.podchaser.com/podcasts/i-rather-be-in-paris-759851/reviews/20617'],['https://t.co/tmiv885WOU'],1,1,...,3132,https://twitter.com/stephfuccio,False,<geotext.geotext.GeoText object at 0x7f784e3adfd0>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/i-r

In [11]:
len(podchaser_reviews[podchaser_reviews['outlinks_cleaned'].str.contains("podcasts/0/")])


30

In [12]:
easy_podchaser = podchaser_reviews[~podchaser_reviews['outlinks_cleaned'].str.contains("podcasts/0/")]

In [13]:
easy_podchaser.head()

,url_x,date,content,renderedContent,id_x,user,outlinks,tcooutlinks,replyCount,retweetCount,...,friendsCount,url_y,verified,geotext,city,country,name,iso3,outlinks_cleaned,podcast_url
0,https://twitter.com/yeoldecrimepod/status/1436049954448060425,2021-09-09 19:32:44+00:00,"It's #PodRevDay, so do the indie podcasters in your life a solid and leave them a review, like t...","It's #PodRevDay, so do the indie podcasters in your life a solid and leave them a review, like t...",1436049954448060425,"{'username': 'yeoldecrimepod', 'displayname': 'Ye Olde Crime Podcast', 'id': 1260623741316403201...",['https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785/reviews/70541'],['https://t.co/zevhkKQBg3'],1,2,...,2870,https://twitter.com/yeoldecrimepod,False,<geotext.geotext.GeoText object at 0x7f784e3eca60>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785/reviews/70541,https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785
2,https://twitter.com/yeoldecrimepod/status/1436022344884043779,2021-09-09 17:43:01+00:00,It's #PodRevDay! Join me in spreading the love to all the great indie podcasts out there. Here's...,It's #PodRevDay! Join me in spreading the love to all the great indie podcasts out there. Here's...,1436022344884043779,"{'username': 'yeoldecrimepod', 'displayname': 'Ye Olde Crime Podcast', 'id': 1260623741316403201...",['https://www.podchaser.com/podcasts/but-it-was-aliens-1096389/reviews/70534'],['https://t.co/0FFGyPITG2'],1,1,...,2870,https://twitter.com/yeoldecrimepod,False,<geotext.geotext.GeoText object at 0x7f784e3eca60>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/but-it-was-aliens-1096389/reviews/70534,https://www.podchaser.com/podcasts/but-it-was-aliens-1096389
4,https://twitter.com/yeoldecrimepod/status/1435996787257577475,2021-09-09 16:01:28+00:00,It's #PodRevDay! Share some love for your favorite podcasts and fellow podcasters. I left a ⭐️⭐️...,It's #PodRevDay! Share some love for your favorite podcasts and fellow podcasters. I left a ⭐️⭐️...,1435996787257577475,"{'username': 'yeoldecrimepod', 'displayname': 'Ye Olde Crime Podcast', 'id': 1260623741316403201...",['https://www.podchaser.com/podcasts/the-secret-sits-1603491/reviews/70522'],['https://t.co/GTaeQZlSlU'],1,2,...,2870,https://twitter.com/yeoldecrimepod,False,<geotext.geotext.GeoText object at 0x7f784e3eca60>,NaN,NaN,NaN,NaN,https://www.podchaser.com/podcasts/the-secret-sits-1603491/reviews/70522,https://www.podchaser.com/podcasts/the-secret-sits-1603491
14,https://twitter.com/ItTakesTwo_pod/status/1435820983806357506,2021-09-09 04:22:53+00:00,Exciting! This is the first #PodRevDay I've seen so not sure if it's weekly/monthly but if anyon...,Exciting! This is the first #PodRevDay I've seen so not sure if it's weekly/monthly but if anyon...,1435820983806357506,"{'username': 'ItTakesTwo_pod', 'displayname': 'It Takes Two', 'id': 1412227274099683331, 'descri...","['https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https://twitter.com/Podchaser/statu...","['https://t.co/yUfYGh2fIO', 'https://t.co/5iCklvxbon']",2,0,...,191,https://twitter.com/ItTakesTwo_pod,False,<geotext.geotext.GeoText object at 0x7f784e3ec6d0>,NaN,New Zealand,New Zealand,NZL,"https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https://twitter.com/Podchaser/status/...","https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https:"
15,https://twitter.com/OracleofDusk/status/1435818285325508613,2021-09-09 04:12:10+00:00,34th (!) #PodRevDay review is for the newest @WhatsHerNamePC episode. \n\nCheck out my ⭐️⭐️⭐️⭐️⭐...,34th (!) #PodRevDay review is for the newest @WhatsHerNamePC episode. \n\nCheck out my ⭐️⭐️⭐️⭐️⭐...,1435818285325508613,"{'username': 'OracleofDusk', 'displayname': 'MJ Bailey', 'id': 1087100443761758208, 'description...",['https://www.podchaser.com/podcasts/whatshername-594083/episodes/the-rani-of-jhansi-lakshmibai-...,['https://t.co/o45ojxkTRX'],1,2,...,2084,https://twitter.com/Or

In [14]:
test = easy_podchaser.iloc[0, -1]
test

'https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785'

In [15]:
url = test
page = httpx.get(url)

In [16]:
page

<Response [200 OK]>

In [17]:
# headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
#             'Referer': 'https://www.google.com'
# }
# page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [18]:
results = soup.find(id='app')
soup.find('a', class_="[object Object]").get_text()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [19]:
soup.find('div', class_='_nymwu9').get_text()

'A weekly Spotify exclusive  podcast'

In [20]:
test2 = easy_podchaser.iloc[1, -1]
test2

'https://www.podchaser.com/podcasts/but-it-was-aliens-1096389'

In [21]:
def get_description(url):
    page = httpx.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    description = soup.find('div', class_='_nymwu9').get_text()

    return description

In [22]:
get_description(test2)

'A weekly Paranormal, Comedy and Astronomy podcast featuring Kevin John and Granville Walker '

In [23]:
test3 = easy_podchaser.iloc[-1, -1]
test3

'https://www.podchaser.com/podcasts/i-rather-be-in-paris-759851'

In [24]:
get_description(test3)

'A Society and Culture podcast'

In [25]:
def get_description(url):
    page = httpx.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    description = soup.find('div', class_='_nymwu9').get_text()

    return description

In [26]:
page3 = httpx.get(test3)

In [27]:
soup = BeautifulSoup(page3.content, 'html.parser')

In [28]:
description = soup.findAll('a', class_="[object Object]")
title = soup.find('h1', class_='_tn1787').get_text() 

In [29]:
soup.find(id="page-about-section").get_text()


AttributeError: 'NoneType' object has no attribute 'get_text'

In [30]:
title

'I Rather Be In Paris'

In [31]:
[d.get_text() for d in description]

['Society', 'Culture']

In [32]:
def get_categories(url):
    page = httpx.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    description = soup.findAll('a', class_="[object Object]")
    categories = [d.get_text() for d in description]
    # title = soup.find('h1', class_='_tn1787').get_text() 

    return categories

In [33]:
def get_titles(url):
    try: 
        page = httpx.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        title = soup.find('h1', class_='_tn1787').get_text()
        
        return title
    except:
        return "ERROR"

In [34]:
get_titles(test3)

'I Rather Be In Paris'

In [41]:
get_categories(test3)

['Society', 'Culture']

In [43]:
print(title)
print(categories)

I Rather Be In Paris


NameError: name 'categories' is not defined

In [36]:
easy_podchaser['title'] = easy_podchaser['podcast_url'].apply(lambda x: get_titles(x))

/tmp/ipykernel_6336/1348602558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  easy_podchaser['title'] = easy_podchaser['podcast_url'].apply(lambda x: get_titles(x))


In [42]:
easy_podchaser['categories'] = easy_podchaser['podcast_url'].apply(lambda x: get_categories(x))

ReadTimeout: The read operation timed out

In [38]:
easy_podchaser.to_csv('data/easy_podchaser_sept.csv', index=False)

In [39]:
easy_podchaser.iloc[:5, -5: ].head()

,name,iso3,outlinks_cleaned,podcast_url,title
0,NaN,NaN,https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785/reviews/70541,https://www.podchaser.com/podcasts/malice-after-midnight-a-true-c-1966785,Malice After Midnight: A True Crime Podcast
2,NaN,NaN,https://www.podchaser.com/podcasts/but-it-was-aliens-1096389/reviews/70534,https://www.podchaser.com/podcasts/but-it-was-aliens-1096389,But It Was Aliens
4,NaN,NaN,https://www.podchaser.com/podcasts/the-secret-sits-1603491/reviews/70522,https://www.podchaser.com/podcasts/the-secret-sits-1603491,The Secret Sits
14,New Zealand,NZL,"https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https://twitter.com/Podchaser/status/...","https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https:",ERROR
15,NaN,NaN,https://www.podchaser.com/podcasts/whatshername-594083/episodes/the-rani-of-jhansi-lakshmibai-97...,https://www.podchaser.com/podcasts/whatshername-594083,What'sHerName


In [40]:
easy_podchaser[easy_podchaser['title']=='ERROR']

,url_x,date,content,renderedContent,id_x,user,outlinks,tcooutlinks,replyCount,retweetCount,...,url_y,verified,geotext,city,country,name,iso3,outlinks_cleaned,podcast_url,title
14,https://twitter.com/ItTakesTwo_pod/status/1435820983806357506,2021-09-09 04:22:53+00:00,Exciting! This is the first #PodRevDay I've seen so not sure if it's weekly/monthly but if anyon...,Exciting! This is the first #PodRevDay I've seen so not sure if it's weekly/monthly but if anyon...,1435820983806357506,"{'username': 'ItTakesTwo_pod', 'displayname': 'It Takes Two', 'id': 1412227274099683331, 'descri...","['https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https://twitter.com/Podchaser/statu...","['https://t.co/yUfYGh2fIO', 'https://t.co/5iCklvxbon']",2,0,...,https://twitter.com/ItTakesTwo_pod,False,<geotext.geotext.GeoText object at 0x7f784e3ec6d0>,NaN,New Zealand,New Zealand,NZL,"https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https://twitter.com/Podchaser/status/...","https://www.podchaser.com/podcasts/it-takes-two-1969712', 'https:",ERROR
110,https://twitter.com/HHSConspiracy/status/1435590133835829248,2021-09-08 13:05:34+00:00,Hey all it's #podrevday a day to celebrate and review those podcasts that have found their way i...,Hey all it's #podrevday a day to celebrate and review those podcasts that have found their way i...,1435590133835829248,"{'username': 'HHSConspiracy', 'displayname': 'Hush Hush Society Conspiracy Hour', 'id': 12828416...","['https://www.podchaser.com/podcasts/hush-hush-society-conspiracy-h-1369558', 'http://www.hushhu...","['https://t.co/myc2HGU9SF', 'https://t.co/r13zfFleXi', 'https://t.co/jBXPESABFU']",1,1,...,https://twitter.com/HHSConspiracy,False,<geotext.geotext.GeoText object at 0x7f784e3ad850>,Los Angeles,United States,United States,USA,"https://www.podchaser.com/podcasts/hush-hush-society-conspiracy-h-1369558', 'http://www.hushhush...","https://www.podchaser.com/podcasts/hush-hush-society-conspiracy-h-1369558', 'http:",ERROR
113,https://twitter.com/CalliopicProd/status/1435584114179153920,2021-09-08 12:41:39+00:00,It's #podrevday why not leave us your thoughts?\n\nVoidless - https://t.co/BQBtGNtLLS\n#WKNPod -...,It's #podrevday why not leave us your thoughts?\n\nVoidless - podchaser.com/podcasts/voidl…\n#WK...,1435584114179153920,"{'username': 'CalliopicProd', 'displayname': 'Calliopic Productions', 'id': 1237802328922763271,...","['https://www.podchaser.com/podcasts/voidless-1304391', 'https://www.podchaser.com/podcasts/we-k...","['https://t.co/BQBtGNtLLS', 'https://t.co/qTKBm8viuH']",1,3,...,https://twitter.com/CalliopicProd,False,<geotext.geotext.GeoText object at 0x7f784e3ad100>,NaN,NaN,NaN,NaN,"https://www.podchaser.com/podcasts/voidless-1304391', 'https://www.podchaser.com/podcasts/we-kno...","https://www.podchaser.com/podcasts/voidless-1304391', 'https:",ERROR
114,https://twitter.com/Podchaser/status/1435583459708137475,2021-09-08 12:39:03+00:00,10 Podcasters Explain Why They Ask for Reviews – In honor of #PodRevDay ❤ \n\nhttps://t.co/su82D...,10 Podcasters Explain Why They Ask for Reviews – In honor of #PodRevDay ❤ \n\npodchaser.com/arti...,1435583459708137475,"{'username': 'Podchaser', 'displayname': 'Podchaser', 'id': 772231864937361408, 'description': '...",['https://www.podchaser.com/articles/community/10-podcasters-explain-why-they-ask-for-reviews'],['https://t.co/su82DN1OHh'],0,3,...,https://twitter.com/Podchaser,False,<geotext.geotext.GeoText object at 0x7f784e2a0070>,NaN,NaN,NaN,NaN,https://www.podchaser.com/articles/community/10-podcasters-explain-why-they-ask-for-reviews,https://www.podchaser.com/articles/community,ERROR
164,https://twitter.com/ODParlayHour/status/1432689133965688832,2021-08-31 12:58:02+00:00,"#PODRAID with @aPODalypseGroup! #Download, #review &amp; #share the one &amp; only @ItGoDowninth...","#PODRAID with @aPODalypseGroup! #Download, #review &amp; #share the one &amp; only @ItGoDowninth...",1432689133965688832,"{'username': 'ODParlayHour', 'displayname': 'ODP